# Πειραματισμοί

In [85]:
from numpy import *
from numpy.linalg import *
from filling import *
from transforms_projections import *

data = load('h3.npy', allow_pickle=True).tolist()
for (key, value) in data.items():
    exec(key + ' = value')
data.keys()

dict_keys(['verts', 'vertex_colors', 'face_indices', 'depth', 'cam_eye', 'cam_up', 'cam_lookat', 'ka', 'kd', 'ks', 'n', 'light_positions', 'light_intensities', 'Ia', 'M', 'N', 'W', 'H', 'bg_color'])

# Φωτισμός και Υλικό Επιφάνειας

## A1. Ambient Light

In [8]:
def ambient_light(ka, Ia):
    I = ka * Ia
    return I

## A2. Diffuse Light

In [135]:
def diffuse_light(P, N, color, kd, light_positions, light_intensities):
    L = (light_positions - P) / norm(light_positions - P, axis=0)
    I = sum(kd * light_intensities * maximum(0.0, N.T @ L), axis=1).reshape(-1, 1)
    return I * color

## Specular Light

In [136]:
def specular_light(P, N, color, cam_pos, ks, n, light_positions, light_intensities):
    L = (light_positions - P) / norm(light_positions - P, axis=0)
    R = maximum(0.0, 2 * N.T @ L) * N - L
    V = (cam_pos - P) / norm(cam_pos - P, axis=0)
    I = sum(ks * light_intensities * (maximum(0.0, V.T @ R) ** n), axis=1).reshape(-1, 1)
    return I * color

In [141]:
vertices = verts.T.copy()
face_indices = face_indices.T.copy()

In [ ]:
surface_normals = cross(vertices[:, face_indices[1]] - vertices[:, face_indices[0]],
                        vertices[:, face_indices[2]] - vertices[:, face_indices[0]],
                        axis=0)

surface_normals /= norm(surface_normals, axis=0)

for vertex in range(vertices.shape[1]):
    N_k = sum(surface_normals[:, where(face_indices == 0)[1]], axis=1).reshape(-1 ,1)
    N_k /= norm(N_k, axis=0)
    

# B. Shading

## Β.1 Υπολογισμός κανονικών διανυσμάτων επιφάνειας

In [6]:
def calculate_normals(vertices, face_indices):
    surface_normals = cross(vertices[:, face_indices[1]] - vertices[:, face_indices[0]],
                            vertices[:, face_indices[2]] - vertices[:, face_indices[0]],
                            axis=0)

    surface_normals /= norm(surface_normals, axis=0)


## Β2. Συνάρτηση φωτογράφισης

In [ ]:
def render_object(shader,
                  focal, eye, lookat, up, bg_color,
                  M, N, H, W,
                  verts, vert_colors, face_indices,
                  ka, kd, ks, n, light_position, light_intensities, Ia):
    # shader : string, 'gouraud' or 'phong'
    # focal : scalar, the f in the previous assignment
    # eye : 3x1, the camera's coordinates
    # lookat : 3x1, where the camera is looking coordinates
    # up : 3x1, camera's pose
    # bg_color : 3x1, RGB of background
    # M, N : scalar, image's resolution
    # H, W : camera's lens
    # verts : 3xN_u : all corner's coordinates
    # verts_color: 3xN_u : colors of all corners
    # 
    pass

## B3. Gouraud Shading

In [ ]:
def shade_gouraud(verts_p, verts_n, verts_c, bcoords, cam_pos, ka, kd, ks, n, light_positions, light_intensities, Ia, X):
    # verts_p : 2x3, 2D coordinates of triangle's corners after projection
    # verts_n : 3x3, normal vectors of triangle's corners
    # verts_c : 3x3, color of triangle's corners

    # bcoords : 3x1, center of gravity
    # cam_pos : 3x1, coordinates of camera
    # ka, kd, ks, n : scalars, illumination constants
    # light_positions : 3x(# of sources), coordinates of light sources
    # light_intensities : 3x(# of sources), intensities of these light sources
    # Ia : 3x1, ambient light constant
    # X : MxNx3, image before
    # Y : MxNx3, image after
    pass

## B4. Phong Shading

In [ ]:
def shade_phong(verts_p, verts_n, verts_c, bcoords, cam_pos, ka, kd, ks, n, light_positions, light_intensities, Ia, X):
    # verts_p : 2x3, 2D coordinates of triangle's corners after projection
    # verts_n : 3x3, normal vectors of triangle's corners
    # verts_c : 3x3, color of triangle's corners

    # bcoords : 3x1, center of gravity
    # cam_pos : 3x1, coordinates of camera
    # ka, kd, ks, n : scalars, illumination constants
    # light_positions : 3x(# of sources), coordinates of light sources
    # light_intensities : 3x(# of sources), intensities of these light sources
    # Ia : 3x1, ambient light constant
    # X : MxNx3, image before
    # Y : MxNx3, image after
    pass